In [44]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import pickle
from sklearn.metrics import root_mean_squared_error

In [46]:
########### CONNECT TO ThalesStockPredictor SQLServer DB

# Define connection string
connStr = 'mssql+pyodbc://@MSI/ThalesStockPredictor?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'

# Create SQLAlchemy engine
engine = create_engine(connStr)

# Define the query
query = """
SELECT * FROM vw_COMBINED_MODEL
ORDER BY FK_DT_Date desc
"""

# Execute the query and assign the result to a pandas DataFrame
Model_Data = pd.read_sql(query, engine)

# Close the SQL Server Connection
engine.dispose()

In [47]:
# Prep Model_Data DF for splitting Train/Test

# copy Model_Data to Model_Norm for ML steps
Model_Norm = Model_Data.copy()

# change date column to be number date (UNIX Epoch)
Model_Norm['FK_DT_Date'] = pd.to_datetime(Model_Norm['FK_DT_Date']).astype('int64') // 10**9

# remove NA's which will also remove na' from lagged y predictor THA_NextDay_Close column
Model_Norm = Model_Norm.dropna()

In [48]:
Model_Norm

,FK_DT_Date,THA_Open,THA_High,THA_Low,THA_Close,THA_Adj_Close,THA_Volume,SPI_Open,SPI_High,SPI_Low,...,SPI_UpperBand,SPI_LowerBand,FRA_SMA,FRA_UpperBand,FRA_LowerBand,THA_OBV,EUR_OBV,SPI_OBV,FRA_OBV,THA_NextDay_Close
1,1709510400,136.000000,138.600006,135.850006,138.350006,138.350006,351633.0,5130.990234,5149.669922,5127.180176,...,5150.409361,4913.836537,7798.610010,8061.207936,7536.012083,20722978.0,2.318764e+10,1.296771e+12,8.148700e+09,150.899994
4,1709078400,137.000000,137.149994,135.850006,136.600006,136.600006,186668.0,5067.200195,5077.370117,5058.350098,...,5126.106344,4872.754496,7746.257983,8014.985481,7477.530486,19622658.0,2.300034e+10,1.291562e+12,8.161566e+09,137.050003
5,1708992000,138.899994,139.100006,136.000000,136.850006,136.850006,254221.0,5074.600098,5080.689941,5057.290039,...,5118.492300,4865.889585,7731.375977,7984.059230,7478.692723,19809326.0,2.294370e+10,1.295351e+12,8.108808e+09,136.600006
6,1708905600,135.500000,139.250000,135.050003,138.750000,138.750000,304159.0,5093.000000,5097.660156,5068.910156,...,5107.263494,4862.093391,7717.829492,7949.717954,7485.941031,20063547.0,2.288397e+10,1.291425e+12,8.058411e+09,136.850006
9,1708473600,135.300003,135.300003,132.649994,134.850006,134.850006,238055.0,4963.029785,4983.209961,4946.000000,...,5063.497564,4846.691400,7664.382007,7830.537080,7498.226933,19747860.0,2.266789e+10,1.287385e+12,7.983094e+09,134.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6207,947721600,36.000000,36.750000,34.000000,34.000000,19.221630,519684.0,1432.250000,1454.199951,1432.250000,...,1478.000940,1384.167951,5614.952257,5887.180144,5342.724370,-890813.0,-3.567400e+06,2.500600e+09,0.000000e+00,37.400002
6208,947635200,33.549999,35.139999,33.549999,34.889999,19.724779,186958.0,1438.560059,1442.599976,1427.079956,...,1476.648518,1380.871471,5612.666260,5903.320486,5322.012034,-371129.0,-5.168360e+07,1.470200e+09,0.000000e+00,34.000000
6209,947548800,35.189999,35.189999,33.750000,35.000000,19.786970,205261.0,1457.599976,1458.660034,1434.420044,...,1479.897156,1376.625688,5616.975725,5929.811982,5304.139469,-184171.0,-8.636670e+07,2.444800e+09,0.000000e+00,34.889999
6210,947462400,35.500000,35.490002,34.700001,35.200001,19.900040,201905.0,1441.469971,1464.359985,1441.469971,...,1482.227186,1370.862780,5617.488363,5960.170428,5274.806297,21090.0,-4.247670e+07,3.458800e+09,0.000000e+00,35.000000


In [49]:
# Split dataframe into Training, Validation, Testing before normalization

# Proportion for training set
train_prop = 0.9

# Create training and testing set
training_data, testing_data = train_test_split(Model_Norm, test_size=1-train_prop, random_state=123)

In [10]:
training_data

,FK_DT_Date,THA_Open,THA_High,THA_Low,THA_Close,THA_Adj_Close,THA_Volume,SPI_Open,SPI_High,SPI_Low,...,SPI_UpperBand,SPI_LowerBand,FRA_SMA,FRA_UpperBand,FRA_LowerBand,THA_OBV,EUR_OBV,SPI_OBV,FRA_OBV,THA_NextDay_Close
109,1695945600,133.449997,134.449997,131.750000,133.100006,132.320511,227405.0,4328.180176,4333.149902,4274.859863,...,4577.855816,4240.334174,7220.757983,7389.365124,7052.150843,21782717.0,2.188945e+10,1.205140e+12,7.067289e+09,131.850006
2936,1347408000,27.160000,27.770000,26.799999,27.635000,21.387466,192050.0,1433.560059,1439.150024,1432.989990,...,1439.561184,1391.549815,3463.260510,3560.834720,3365.686300,-9432575.0,9.137304e+09,5.892957e+11,-1.967040e+09,26.620001
3502,1277942400,26.235001,26.385000,25.879999,26.040001,19.156347,752089.0,1031.099976,1033.579956,1010.909973,...,1136.946472,1021.992505,3552.648499,3793.996812,3311.300185,-12745656.0,8.301596e+09,3.341313e+11,-1.355651e+09,26.330000
3501,1278028800,26.330000,26.520000,25.915001,26.330000,19.369684,445916.0,1027.650024,1032.949951,1015.929993,...,1139.974835,1014.734143,3547.286499,3802.101592,3292.471406,-12299740.0,8.197605e+09,3.301628e+11,-1.194216e+09,25.799999
839,1606435200,79.540001,80.699997,78.879997,79.860001,74.154907,327069.0,3638.550049,3644.310059,3629.330078,...,3742.298902,3320.519066,5326.550537,5892.033722,4761.067352,801386.0,2.123497e+10,1.244042e+12,5.366041e+09,77.059998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5628,1017792000,40.000000,40.500000,39.669998,40.220001,23.360634,529742.0,1136.760010,1138.849976,1119.680054,...,1178.226182,1127.049831,4602.704468,4678.058184,4527.350752,-2409532.0,-8.384587e+08,-2.774320e+10,0.000000e+00,40.730000
4369,1170806400,40.500000,40.869999,40.240002,40.680000,27.136269,386987.0,1447.410034,1452.989990,1446.439941,...,1453.988168,1411.307816,5622.001465,5706.366956,5537.635974,1793285.0,8.749449e+09,1.654049e+11,4.202605e+09,40.410000
1452,1531094400,109.949997,111.699997,109.800003,110.500000,96.906342,243767.0,2775.620117,2784.649902,2770.729980,...,2808.236559,2692.753455,5369.949951,5521.725888,5218.174014,21376782.0,1.884260e+10,9.832408e+11,4.000986e+09,111.800003
3728,1250121600,30.020000,30.110001,29.745001,29.745001,21.469219,476751.0,1005.859985,1013.140015,1000.820007,...,1028.545563,941.998431,3408.613000,3588.759213,3228.466788,-11154039.0,9.075627e+09,2.486845e+11,1.266049e+09,29.879999


In [12]:
# XGBoost Model

train_x = training_data.iloc[:,0:84]
train_y = training_data.iloc[:,85]

best_params = {
    'objective': 'reg:squarederror',
    'eta': 0.01,
    'max_depth': 5,
    'gamma': 1,
    'colsample_bytree': 1,
    'min_child_weight': 10,
    'subsample': 0.6
}

dtrain = xgb.DMatrix(train_x, label=train_y)
cv_results = xgb.cv(best_params, dtrain, num_boost_round=5000, nfold=10, metrics='rmse', early_stopping_rounds=10)


In [13]:
# finding the best iteration
best_iteration = cv_results['test-rmse-mean'].idxmin()
print(f"Best iteration: {best_iteration}")

Best iteration: 672


In [16]:
# Assuming testing_data is your test dataset
test_x = testing_data.iloc[:,0:84]
test_y = testing_data.iloc[:,85]

# Train the final model with the optimal number of boosting rounds
final_model = xgb.train(best_params, dtrain, num_boost_round=best_iteration)

# Make predictions on your test data
dtest = xgb.DMatrix(test_x)
test_predictions = final_model.predict(dtest)

In [17]:
# Calculate test RMSE
test_rmse = sqrt(mean_squared_error(test_y, test_predictions))
print(f"Test RMSE: {round(test_rmse, 4)}")

# dataframe with actual and predicted values from bestiteration of XGBoost Model
pred_xgb = pd.DataFrame({'test_y': test_y, 'pred_y': test_predictions})

# Print the xgboost predictions
print(pred_xgb)

Test RMSE: 1.0847
          test_y      pred_y
320   122.400002  121.755341
3393   27.915001   27.384539
4623   38.849998   38.619419
1155  109.699997  109.626884
4705   37.139999   37.356499
...          ...         ...
3948   35.840000   35.947163
1419  117.800003  118.754875
1915   80.300003   80.038254
5627   41.000000   40.518532
4726   38.820000   38.374386

[577 rows x 2 columns]


In [21]:
# Create a KFold object
kf = KFold(n_splits=10, shuffle=True, random_state=123)

# Linear Regression model
lm = make_pipeline(StandardScaler(), LinearRegression())
lm_scores = cross_val_score(lm, training_data.drop('THA_NextDay_Close', axis=1), training_data['THA_NextDay_Close'], cv=kf, scoring='neg_root_mean_squared_error')
lm_rmse = -lm_scores.mean()
print(f"Linear Regression RMSE: {round(lm_rmse, 4)}")

Linear Regression RMSE: 1.0612


In [22]:
# Random Forest model
rf = make_pipeline(StandardScaler(), RandomForestRegressor(random_state=123))
rf_scores = cross_val_score(rf, training_data.iloc[0:5186].drop('THA_NextDay_Close', axis=1), training_data.iloc[0:5186]['THA_NextDay_Close'], cv=kf, scoring='neg_root_mean_squared_error')
rf_rmse = -rf_scores.mean()
print(f"Random Forest RMSE: {round(rf_rmse, 4)}")

Random Forest RMSE: 1.1006


In [27]:
# Fit a RandomForest model to compute feature importances
rf = RandomForestRegressor(random_state=123)
rf.fit(training_data.iloc[:, 0:85], training_data.iloc[:, 85])

# Get feature importances
importances = rf.feature_importances_

# Create a DataFrame of features and their importances
importance_df = pd.DataFrame({'Feature': training_data.columns[0:85], 'Importance': importances})

# Sort the DataFrame by importance
importance_df = importance_df.sort_values('Importance', ascending=False)

# Print the DataFrame
importance_df

          Feature  Importance
4       THA_Close    0.905067
2        THA_High    0.060403
5   THA_Adj_Close    0.024881
3         THA_Low    0.006691
1        THA_Open    0.001173
..            ...         ...
14       FRA_High    0.000004
31      SPI_EMA12    0.000004
23  EUR_Adj_Close    0.000004
20       EUR_High    0.000004
15        FRA_Low    0.000004

[85 rows x 2 columns]


In [28]:
# Get the names of the top 60 features
top_60_features = importance_df['Feature'].iloc[:60]

# Create a new DataFrame with only the top 60 features
training_data_important = training_data[top_60_features]

In [29]:
# Fit a new RandomForest model on the important features
rf_important = RandomForestRegressor(random_state=123)
rf_important.fit(training_data_important, training_data['THA_NextDay_Close'])

RandomForestRegressor(random_state=123)

In [33]:
# Make predictions on the training data
train_predictions = rf_important.predict(training_data_important)

# Calculate training RMSE
train_rmse = root_mean_squared_error(training_data.iloc[:, 85], train_predictions)
print(f"Training RMSE: {round(train_rmse, 4)}")

Training RMSE: 0.4077


In [ ]:
# Save the RandomForest model
with open('rfModel.pkl', 'wb') as file:
    pickle.dump(rf_important, file)

# Load the saved model
with open('rfModel.pkl', 'rb') as file:
    loaded_rf = pickle.load(file)

# # Use the loaded model to make predictions on the test data
# loaded_predictions = loaded_rf.predict(testing_data_important)
